# PoE Harvest Simulation
This notebook simulates harvest plots. The goal is to determine the breakpoint where Crop Rotation is (on average) better non-Crop-Rotation harvest.

Baseline chances have taken from https://forgottenarbiter.github.io/Poe-Harvest-Mechanics/ - both for regular harvest and for crop rotation.

The most interesting part is that the T3 harvest mob chance follows a binomial distribution with p=0.25, which can then be increased through through idols or the atlas passive tree. And that Crop Rotation upgrade chance seems to scale with map tier, which could be especially interesting for T17 harvest farming.



In [17]:
import random
import numpy as np


np.set_printoptions(suppress=True)
sample_amount = 1_000_000

In [18]:

def sample(n, p):

    s1 = random.binomialvariate(n, p)
    s2 = random.binomialvariate(n, p)

    return max(s1, s2)


print(np.mean([sample(3, 0.25) for i in range(sample_amount)]))
print(np.mean([sample(3, 0.25*1.3) for i in range(sample_amount)]))

1.141552
1.407877


The above shows that the average T3 per pair of plots (of which you can only take one because the other one wilts) is 1.14 T3 mobs for T16 harvest without atlas tree investment, and 1.4 T3 mobs if you do specced into it.

In [28]:
def sample_crop_rotation(population = [23, 0, 0, 0], chances = [0.25, 0.2, 0.05, 0], plots=9):

    def upgrade(pop, chances):
        
        upgraded = []
        for i, n in enumerate(pop):
            #if i == len(pop)-1:
            #    continue
            #result[i] = pop[i]
            count = 0
            for _ in range(n):
                if random.uniform(0,1)<chances[i]:
                    count += 1
            upgraded.append(count)
        
        result = []
        for i, n in enumerate(pop):
            if i == 0:
                result.append(pop[i]-upgraded[i])
                continue
            if i == len(pop)-1:
                result.append(pop[i]+upgraded[i-1])
                continue
            result.append(pop[i]-upgraded[i]+upgraded[i-1])

        return result

    #chances = [0.25, 0.2, 0.05, 0]

    #population = [23, 0, 0, 0]

    res = [population]
    for i in range(plots):
        res.append(upgrade(res[-1], chances))

    return res

total = np.array(sample_crop_rotation())

for _ in range(sample_amount-1):
    total = np.add(total, np.array(sample_crop_rotation()))

means = np.divide(total, sample_amount)
print(means)

[[23.        0.        0.        0.      ]
 [17.245471  5.754529  0.        0.      ]
 [12.937553  8.911344  1.151103  0.      ]
 [ 9.70272  10.365331  2.875003  0.056946]
 [ 7.279218 10.715935  4.803956  0.200891]
 [ 5.458165 10.394039  6.706392  0.441404]
 [ 4.093292  9.678415  8.451611  0.776682]
 [ 3.069545  8.767047  9.963751  1.199657]
 [ 2.300624  7.783913 11.216916  1.698547]
 [ 1.724642  6.804545 12.211602  2.259211]]


The above estimates the distribution of T1, T2, T3, T4 mobs after each upgrade step, based on T16 harvest data. 
Interestingly, after only 2 upgrade steps the following plots have at least as many T3 mobs as regular non-specced harvest has per plot pair.
After 5 upgrade steps, the next plot alone has almost as many T3 mobs as a non-crop-rotation but specced harvest has with its maximum amount of 5 plots in total (6.7 vs 7 respectively).

In [27]:
def cumul(indices, 
          distribution=[[23.,        0.,        0.,        0.      ],
                        [17.250176,  5.749824,  0.,        0.      ],
                        [12.938635,  8.912266,  1.149099,  0.      ],
                        [ 9.702046, 10.368255,  2.872332,  0.057367],
                        [ 7.274691, 10.722638,  4.801632,  0.201039],
                        [ 5.457264, 10.395335,  6.706354,  0.441047],
                        [ 4.092606,  9.68018,   8.451636,  0.775578],
                        [ 3.069959,  8.766407,  9.965851,  1.197783],
                        [ 2.302701,  7.779758, 11.222836,  1.694705],
                        [ 1.727466,  6.798853, 12.219247,  2.254434]],
            ):
    
    
    out = np.zeros(4)
    for i in indices:
        out += distribution[i]
    return out

print("3 plots:")

indices = [0, 1, 2]
print(cumul(indices))

indices = [0, 0, 2]
print(cumul(indices))

indices = [0, 1, 1]
print(cumul(indices))


print("4 plots:")

indices = [0, 1, 2, 2]
print(cumul(indices))

indices = [0, 1, 1, 3]
print(cumul(indices))

indices = [0, 0, 2, 2]
print(cumul(indices))

indices = [0, 0, 0, 3]
print(cumul(indices))

print("5 plots:")

indices = [0, 0, 2, 2, 4]
print(cumul(indices))

indices = [0, 0, 0, 3, 3]
print(cumul(indices))

indices = [0, 1, 1, 1, 4]
print(cumul(indices))

indices = [0, 0, 0, 3, 4]
print(cumul(indices))

print("6 plots:")

indices = [0, 0, 0, 3, 3, 5]
print(cumul(indices))

indices = [0, 0, 2, 2, 2, 5]
print(cumul(indices))

indices = [0, 0, 0, 3, 4, 4]
print(cumul(indices))

indices = [0, 0, 2, 2, 4, 4]
print(cumul(indices))

indices = [0, 1, 1, 2, 4, 4]
print(cumul(indices))

# R Y B B Y R
indices = [0, 0, 1, 1, 3, 4]
print(cumul(indices))

indices = [0, 0, 0, 3, 3, 3]
print(cumul(indices))

print("7 plots:")
indices = [0, 0, 0, 3, 3, 5, 5]
print(cumul(indices))

indices = [0, 0, 2, 2, 4, 4, 4]
print(cumul(indices))

indices = [0, 0, 2, 2, 2, 5, 5]
print(cumul(indices))

indices = [0, 0, 0, 0, 4, 4, 6]
print(cumul(indices))

indices = [0, 0, 2, 2, 2, 2, 6]
print(cumul(indices))

indices = [0, 1, 1, 1, 1, 4, 4]
print(cumul(indices))

indices = [0, 2, 2, 3, 3, 3, 3]
print(cumul(indices))

3 plots:
[53.188811 14.66209   1.149099  0.      ]
[58.938635  8.912266  1.149099  0.      ]
[57.500352 11.499648  0.        0.      ]
4 plots:
[66.127446 23.574356  2.298198  0.      ]
[67.202398 21.867903  2.872332  0.057367]
[71.87727  17.824532  2.298198  0.      ]
[78.702046 10.368255  2.872332  0.057367]
5 plots:
[79.151961 28.54717   7.09983   0.201039]
[88.404092 20.73651   5.744664  0.114734]
[82.025219 27.97211   4.801632  0.201039]
[85.976737 21.090893  7.673964  0.258406]
6 plots:
[93.861356 31.131845 12.451018  0.555781]
[90.273169 37.132133 10.153651  0.441047]
[93.251428 31.813531 12.475596  0.459445]
[86.426652 39.269808 11.901462  0.402078]
[84.988369 41.85719  10.752363  0.402078]
[97.477089 32.590541  7.673964  0.258406]
[98.106138 31.104765  8.616996  0.172101]
7 plots:
[99.31862  41.52718  19.157372  0.996828]
[93.701343 49.992446 16.703094  0.603117]
[95.730433 47.527468 16.860005  0.882094]
[110.641988  31.125456  18.0549     1.177656]
[101.847146  45.329244  13.

The cumulative function above displays a couple of strategies to harvest crop rotation, depending on how many plots you have. Generally, the strategy to do the colors in decending order of availability at the start seems to give best results.